In [6]:
"""
Script to demonstrate the usage of shared dicts using multiple workers.

In the first epoch the shared dict in the dataset will be filled with
random values. The next epochs will just use the dict without "loading" the
data again.

@author: ptrblck
"""

from multiprocessing import Manager
from clip_retrieval.clip_client import ClipClient, Modality
import torch
from torch.utils.data import Dataset, DataLoader
client_backup2 = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=500,
    deduplicate=False,
)

class CaptionCache(Dataset):
    def __init__(self, shared_dict, length):
        self.shared_dict = shared_dict
        self.length = length

    def __getitem__(self, index, imagepath, K = None):
        if index not in self.shared_dict:
            print('Adding {} to shared_dict'.format(index))
            res = client_backup2.query(image=imagepath)
            self.shared_dict[index] = [each['caption'] for each in res[:self.length]]
            assert len(self.shared_dict[index]) == K ## type list
        else:
            print("use cache")
        return self.shared_dict[index][:K]

    def __len__(self):
        return self.length


# Init
manager = Manager()
shared_dict = manager.dict()
dataset = CaptionCache(shared_dict, length=128)

print(dataset.shared_dict)
captions = dataset.__getitem__(0, '/data/seongha/caltech-101/101_ObjectCategories/accordion/image_0001.jpg', 128)
assert len(captions) == 128

captions = dataset.__getitem__(0, '/data/seongha/caltech-101/101_ObjectCategories/accordion/image_0001.jpg', 8)
assert len(captions) == 8
print(captions)

{}
Adding 0 to shared_dict


use cache
['Watkins Musette Accordion bellows open view', 'accordion', 'Hohner Fun Top 120', 'The Childrens Museum of Indianapolis - Titano accordion.jpg', 'Piano Accordion Vector Clipart image.', 'German Barcarole Magister, 80 Bass, 8 Registers, Case, Piano Accordion Instrument (485)', 'The Childrens Museum of Indianapolis - Titano accordion.jpg', 'Accordion Lounge']
